# Librería FTPLIB (acceso al Servidor)

_____________________________________________

visto en     https://decodigo.com/cliente-ftp-en-python

búsqueda    python ftp

In [6]:
from ftplib import FTP, all_errors

servidor = 'javicu.synology.me'
usuario = 'invitado'
password = '1234'

### Conexión al servidor

In [7]:
# Conectarse en un solo paso (puerto 21)
with FTP(host=servidor, user=usuario, passwd=password) as ftp:
    print(ftp.getwelcome())

220 NAS FTP server ready.


In [8]:
# Conectarse en pasos separados para definir puerto 29
ftp = FTP()
ftp.connect(servidor, 21)
ftp.login(usuario, password)
print(ftp.getwelcome())

220 NAS FTP server ready.


### Control errores

In [3]:
with FTP(servidor, usuario, password) as ftp:
    try:
        print(ftp.pwd())
    except ftplib.all_errors as e:
        print(f'Error en FTP: {e}')

/


In [ ]:
# Conexión en un paso (sirve solo para puerto predeterminado 21)
with FTP(servidor, usuario, password) as ftp:
    print("Directorio defecto: ", ftp.pwd())  # Muestra la carpeta default
    print("Cambio a directorio video: ", ftp.cwd('video'))
    print("Directorio defecto: ", ftp.pwd())
    files = []
    ftp.dir(files.append)
    for f in files:
        print(f)
    # ftp.mkd('mi_directorio')
    # ftp.rmd('mi_directorio')

### Trabajar con directorios

* **pwd**.- Carpeta por defecto
* **mkd**.- Crear directorio
* **rmd**.- Eliminar directorio
* **cmd**.- Cambiar directorio
* **dir**.- Listar contenido de un directorio
* **rename(‘nombre_anterior.txt’, ‘ nombre_nuevo.txt’)**.- Renombrar fichero
* **delete(fichero)**.- Eliminar fichero 

In [9]:
#  Muestra la carpeta default
print("Directorio defecto: ", ftp.pwd())

#  Lectura del contenido del directorio
files = []
ftp.dir(files.append)
for f in files:
    print(f)

Directorio defecto:  /
-rwxrwxrwx   1 Invitado users              14 Apr 16 17:03 archivo.txt
-rwxrwxrwx   1 Invitado users          231769 Apr 16 17:03 imagen.png
drwxrwxrwx   1 Invitado users             104 Apr 20 08:43 mi_directorio


In [11]:
#  Cambiar a directorio Invitados
ftp.cwd('mi_directorio')

#  Mostrar directorio actual y contenido
print("Directorio: ", ftp.pwd())
files = []
ftp.dir(files.append)
for f in files:
    print(f)

Directorio:  /mi_directorio
-rwxrwxrwx   1 Javi     users           99584 Jul 30  2021 Aunque tenga miedo, hagalo igual.pdf
-rwxrwxrwx   1 Javi     users          638350 Nov 25  2022 Finis Mundi.epub


In [18]:
#  Crear un nuevo directorio
ftp.mkd('mi_directorio')
# ftp.rmd('mi_directorio')

'mi_directorio'

In [ ]:
#  Cambiar a directorio Invitados
ftp.cwd('mi_directorio')

#  Mostrar directorio actual y contenido
print("Directorio: ", ftp.pwd())
files = []
ftp.dir(files.append)
for f in files:
    print(f)

In [10]:
#  Renombrar fichero o directorio
try:
    ftp.rename('archivo1.txt', 'archivo.txt')
except all_errors as error:
    print(f'Error al renombrar archivo en el servidor: {error}')

Error al renombrar archivo en el servidor: 421 Timeout (300 seconds): closing control connection.


In [56]:
#  Eliminar fichero
try:
    ftp.delete('archivo.txt')
except all_errors as error:
    print(f'Error al borrar archivo: {error}') 

In [26]:
#  Cierre conexión
ftp.close()

### Envío de ficheros

* **storlines**.- Envío de ficheros de texto
* **storbinary**.- Envío de ficheros binarios (imágenes, etc.)

In [57]:
# For text or binary file, always use "rb"
with open('archivo.txt', 'rb') as text_file:
    ftp.storlines('STOR archivo.txt', text_file)
with open('imagen.png', 'rb') as image_file:
    ftp.storbinary('STOR imagen.png', image_file)

### Recepción de ficheros, primero obtenemos tamazo (size)

* **sendcmd('TYPE I') y size(fichero)**.- Obtener tamaño de fichero binario
* **sendcmd('TYPE A') y size(fichero)**.- Obtener tamaño de fichero texto
* **retrlines**.- Recibe fichero de texto
* **retrbinary**.- Recibe fichero binario (imágenes, etc.)

In [53]:
#   Obtenemos el tamaño de los ficheros
try:
    ftp.sendcmd('TYPE I')           # Para archivos binarios
    print(ftp.size('imagen.png'))   # obtiene el tamaño de la imagen en el server
except all_errors as error:
    print(f"Error al obtener tamaño de imagen: {error}")
try:
    ftp.sendcmd('TYPE A')           # Para texto
    print(ftp.size('archivo.txt'))
except all_errors as error:
    print(f"Error al obtener tamaño de archivo: {error}")

231769
14


In [54]:
#  Recibir ficheros del Servidor

# Para archivos de texto
with open('nombre_texto_local.txt', 'w') as local_file:  # Abre un archivo de texto localmente para escritura
    response = ftp.retrlines('RETR archivo.txt', local_file.write)
    # Revisa la respuesta
    # https://en.wikipedia.org/wiki/List_of_FTP_server_return_codes
    if response.startswith('226'):  # Transferencia completa
        print('Transferencia completa')
    else:
        print('Error de transferencia. El archivo puede estar incompleto o corrupto.')

# Para archivos binarios
with open('imagen_local.png', 'wb') as local_file:
    ftp.retrbinary('RETR imagen.png', local_file.write)

Transferencia completa


### Comando FTP - LIST, RETR, NLST o MLSD

* **retrlines(LIST)**.- Listar componentes del directorio
* **retrbinary(RETR ...)**.- Descargar archivo (no cierra el fichero)
* **retrlines(NLST)**.- Devuelve listado simple del contenido (sin información de si es directorio)
* **retrlines(MLSD)**.- Devuelve listado con el contenido con formato, campos

In [4]:
ftp.retrlines('LIST') 

-rwxrwxrwx   1 Invitado users              14 Apr 16 17:03 archivo.txt
-rwxrwxrwx   1 Invitado users          231769 Apr 16 17:03 imagen.png
drwxrwxrwx   1 Invitado users               0 Apr 16 16:31 mi_directorio


'226 Transfer complete.'

In [6]:
#  Descarga el fichero archivo.txt del servidor (no cierra el fichero de salida)
ftp.retrbinary('RETR archivo.txt', open('archivo.txt', 'wb').write)

'226 Transfer complete.'

In [7]:
ftp.retrlines('NLST')

archivo.txt
imagen.png
mi_directorio


'226 Transfer complete.'

In [8]:
ftp.retrlines('MLSD')

modify=20230416170348;type=file;unique=30U10A;size=14;UNIX.mode=0777;UNIX.owner=Invitado;UNIX.group=users; archivo.txt
modify=20230416170348;type=file;unique=30U107;size=231769;UNIX.mode=0777;UNIX.owner=Invitado;UNIX.group=users; imagen.png
modify=20230416163133;type=dir;unique=30U105;size=0;UNIX.mode=0777;UNIX.owner=Invitado;UNIX.group=users; mi_directorio


'226 Transfer complete.'